# Univariate Fine-Mapping of Functional (Epigenomic) Data with fSuSiE


The following command performs univariate fine-mapping for functional (epigenomic) data with fSuSiE. 

The TWAS portion makes use of TWAS weights, linkage disequilibrium data and GWAS summary statistics. Preset variants used are taken from the linkage disequilibrium data and used only for TWAS. 

TWAS cross validation tell us which of the four methods (enet, lasso, mrash, SuSiE) are best to use. By default, we limit to under 5000 variants for cross validation. In cross validation, the data is split into five parts. Training is done on four parts, and prediction is done on the fifth. Linear regression is used to assess the results and get r squared and pvalues. 

Fine mapping with SuSiE follows the formulat y=xb+e where x has many highly correlated variables due to linkage disequilibrium. Therefore, true effects (b), are very sparse. The SuSiE wrapper looks for five independent signals in each region to increase convergence speed. However, if five signals are found, then the the upper limit is increased. SuSiE does not allow for the inclusion of covariates. Therefore, covariates are regressed in.



Input:
    
`--genoFile`: path to a text file contatining information on genotype files. For example:
```
#id     #path
21      $PATH/protocol_example.genotype.chr21_22.21.bed
22      $PATH/protocol_example.genotype.chr21_22.22.bed
```
`--phenoFile`: a tab delimited file containing chr, start, end, ID and path for the regions. For example:
```
#chr    start   end     ID      path
chr21   0       14120807        TADB_1297       $PATH/protocol_example.ha.bed.gz
chr21   10840000        16880069        TADB_1298       $PATH/protocol_example.ha.bed.gz
```

`--covFile`: path to a gzipped file containing covariates in the rows, and sample ids in the columns.  
`--customized-association-windows`: a tab delimited file containing chr, start, end, and ID regions. For example:
```
#chr    start   end     ID
chr21   0       14120807        TADB_1297
chr21   10840000        16880069        TADB_1298
```
`--region-name`: if you only wish to analyze one region, then include the ID of a region found in the `customized-association-windows` file

## Overview

1. Run fSuSiE

## Steps

### i. Run the Fine-Mapping and TWAS with SuSiE

In [ ]:
sos run $PATH/mnm_regression.ipynb fsusie \
    --cwd $PATH/fsusie_test/ \
    --name protocol_example_methylation \
    --genoFile $PATH/mwe_data/protocol_data/output/genotype_by_chrom/protocol_example.genotype.chr21_22.genotype_by_chrom_files.txt \
    --phenoFile $PATH/fsusie_test/protocol_example.ha.phenotype_by_region_files.corrected.reformat.txt \
    --covFile $PATH/mwe_data/protocol_data/output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --container oras://ghcr.io/cumc/pecotmr_apptainer:latest \
    --walltime 2h \
    --numThreads 8 \
    --customized-association-windows $PATH/fsusie_test/regions.reformat.txt \
    -c ../scripts/csg.yml -q neurology \
    --save-data \
    --region-name TADB_1298

## Anticipated Results

`protocol_example_methylation.chr21_10840000_16880069.fsusie_mixture_normal_top_pc_weights.rds`:
* For each region of interest, this file contains: 
    1. susie_on_top_pc - ?
    2. twas_weights - for each variant (for enet, lasso and mrash methods). no susie?
    3. twas predictions - for each sample (for enet, lasso, mrash methods)
    4. twas cross validation results - information on the best method. Data is split into five parts
    6. fsusie results - ?
    7. Y coordinates - ?
    8. fsusie summary - ?
    9. total time elapsed
    10. region info - information on the region specified

`protocol_example_methylation.chr21_10840000_16880069.16_marks.dataset.rds`:
* For each gene of interest, contains residuals for each sample and phenotype
* see [pecotmr code](https://github.com/cumc/pecotmr/blob/68d87ca1d0a059022bf4e55339621cbddc8993cc/R/file_utils.R#L461) for description at fsusie uses the `load_regional_functional_data` function, an explanation of the arguments can be found at the similar `load_regional_association_data` function